# SEMA-3D

The MIT License (MIT)

Copyright (c) 2016 AYLIEN

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
import os
# set cuda params
# 'TORCH_HOME'directory will be used to save origenal esm-1v weights
os.environ['TORCH_HOME'] = "../torch_hub"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [2]:
import os
import copy
import math
import json
import pandas as pd
import numpy as np
import pickle
import scipy

from pathlib import Path
from biotite.structure.residues import get_residues

import esm
from esm.data import BatchConverter
from esm.inverse_folding.util import CoordBatchConverter

import torch
from torch.utils.data import Dataset
from torch import nn
from tqdm import tqdm

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction

import sklearn
import sklearn.metrics as metrics
from sklearn.metrics import r2_score, mean_squared_error, auc, plot_precision_recall_curve

In [3]:
def esmStructDataset(pdb_path,chain):
    '''
    Convert PDB-file into dataset format
    
        Parameters:
            pdb_path (Path): path to pdb-file
            chain (str): antigen chain name
        Returns:
            dict (dict): dictionary, where keys are properties of the protein's tertiary structure
    '''
    structure = esm.inverse_folding.util.load_structure(str(pdb_path), chain.upper())
    resi_index = get_residues(structure)[0]
    resi_aa    = get_residues(structure)[1]
    resi_keys     = []
    cn =[]
    binary = []
    for resi_index_,resi_aa_ in zip(get_residues(structure)[0],get_residues(structure)[1]):
        key = (str(resi_aa_),resi_index_)
        cn.append(None)
        binary.append(None)
        resi_keys.append(key)    
    coords, seq = esm.inverse_folding.util.extract_coords_from_structure(structure)
    return {"pdb_id":pdb_path,"seq":seq,"chain":chain,"coords":coords,"cn":cn,"binary":binary,"residues":resi_keys}

In [4]:
class ESM1vForTokenClassification(nn.Module):
    def __init__(self, num_labels = 2):
        super().__init__()
        self.num_labels = num_labels    
        self.esm1v, self.esm1v_alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
        self.classifier = nn.Linear(512, self.num_labels)

    def forward(self, coords, padding_mask, confidence, tokens):
        prev_output_tokens = tokens[:, :-1]
        target = tokens[:, 1:]
        target_padding_mask = (target == alphabet.padding_idx)
        feat, x = self.esm1v.forward(coords, padding_mask, confidence, prev_output_tokens, features_only = True)
        f = feat[0,:,:]
        tt = torch.transpose(feat,1,2)
        logits = self.classifier(tt )
        return SequenceClassifierOutput(logits=logits)    

In [5]:
def loadModel(path = "../models/sema_3d_cn_atom_r1_8.0_r2_18.0_0.pth"):
    model = ESM1vForTokenClassification().cuda()
    model.load_state_dict(torch.load(path))
    return model

In [6]:
model = loadModel()

/mnt/10tb/tshashkova/esm_env3.9/lib/python3.8/site-packages/esm/pretrained.py:172: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [7]:
_, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()

In [8]:
def get_prediction(sema_models, PDB_path, chain):
    struct = esmStructDataset(PDB_path, chain)
    custom_seq = struct["seq"]
    batch = [(struct["coords"], None, custom_seq)]
    batch_converter = CoordBatchConverter(alphabet)
    coords, confidence, strs, tokens, padding_mask = batch_converter(batch)
    with torch.no_grad():
        preds = []
        for sema_model in sema_models:
            pred = sema_model.forward(coords.cuda(),
                                     padding_mask.cuda(),
                                     confidence.cuda(),
                                     tokens.cuda())
            pred = pred.logits[:,:,1].squeeze().detach().cpu().numpy()
            preds.append(pred)
        pred = np.average(np.array(preds),axis=0)
    return pred

In [9]:
sema_predictions = get_prediction([model], "../data/test.pdb", "A")

FileNotFoundError: [Errno 2] No such file or directory: '../data/test.pdb'

In [ ]:
sema_predictions